# Use
After [installing](./Installation.ipynb), `irobotkernel` should function like any other Jupyter Kernel. This guide will cover use of `irobotframework` as both an interactive tool in [JupyterLab](#Lab) which can feed into [command-line](#CLI) tools, such as you might use in continuous integration. Finally, Notebook documents (like this one) can be part of an [nbsphinx](#nbsphinx) documentation suite.

## Lab
First, start JupyterLab. You may also need to activate your environment.

    jupyter lab
    
If you haven't launched JupyterLab before, you should see something like this:

![The JupyterLab Launcher](./_static/screenshots/guide/00_before.png)

Get started by clicking on the `Robot Fram...`(sic) card. You should see a new notebook:

![A Robot Notebook in JupyterLab](./_static/screenshots/guide/01_new.png)

### Write
The heart of the Robot Framework are writing plain-language **[Test Cases](http://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#creating-test-cases)** and **[Tasks](http://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#creating-tasks)**. In the first cell, you can write your first test, like this:

In [ ]:
*** Test Cases ***
Hello, robot!
   Log  Hello, robot!
   Should Be Equal As Numbers  1  1

Clicking the **Log** or **Report** button above should open a new browser tab (for now) of the content:

| Log | Report |
|-----|--------|
| ![](./_static/screenshots/guide/log.png) | ![](./_static/screenshots/guide/report.png) |
| shows a collapsible tree of the suites, tests, tasks and keywords | shows a filterable table of suites of tests/tasks |

#### Tasks
With [Robot Framework 3.1][rf31], a new type of executable activity is available: **Tasks**. These can be used interchangeably with **Test Cases**, but can't be used together in the same Notebook.

[rf31]: https://github.com/robotframework/robotframework/blob/master/doc/releasenotes/rf-3.1.rst

In [ ]:
*** Tasks ***
Hello, robot!
   Log  Hello, robot!
   Should Be Equal As Numbers  1  1

So pretty much pick one or the other!

### Keywords
The reusable pieces of **Tests Cases** and **Tasks** are [**Keywords**](http://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#creating-user-keywords). You can define keywords anywhere in the same cell as where you use them.

In [ ]:
*** Test Cases ***
Have a nice day
    Have a nice day

*** Keywords ***
Have a ${kind of} day
    Log  It is a ${kind of} day!
    Should Not Contain  ${kind of}  bad   msg=I guess it was a ${kind of} day

And now as long as your kernel is running, you'll be able to reuse that keyword:

In [ ]:
*** Test Cases ***
Have a not-so-nice day
    Have a terrible, horrible, no-good, very bad day

### Settings
While you are building an `irobotframework` Notebook, you're actually continuously adding to a **[Suite](http://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#creating-test-suites)**, with **Settings** that propagate down to all the tests... typically, you would create these either in a separate cell _before_ running any tests or tasks.

In [ ]:
*** Settings ***
Documentation   This is my suite.
Force Tags   days

*** Test Cases ***
Have a decent day
    [Tags]  pretty-good
    Have a pretty good day

> _Note that the log now contains your documentation, and the test run has a tag from its **Suite**, as well as it's own. Also, now all test runs from here on out will also have the `days` tag!_

### Libraries
While it's great to write your own keywords in the language of your application and users, following the pattern of python's "batteries included", Robot Framework (and therefore `irobotframework`) includes a number of powerful **[Libraries](http://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#using-test-libraries)**. The keywords like **Log** and **Should not Contain** come from the **[BuiltIn](http://robotframework.org/robotframework/latest/libraries/BuiltIn.html)** library, and are always available, while others need to be imported with a `Library` setting.

In [ ]:
*** Settings ***
Library   OperatingSystem

*** Test Cases ***
How big is this file?
    ${size} =  Get File Size  Guide.ipynb
    Log   ${size / 1024} kilobytes

#### More Libraries
A lot of other libraries (of various quality and states of maintenance) are available from [PyPI](https://pypi.org/search/?q=&c=Framework+%3A%3A+Robot+Framework+%3A%3A+Library). However, you can also create your own, with the `%%python module` magic:

In [ ]:
%%python module MyLibrary
def get_greeting(to_whom: str="World") -> str:
    return f"Hello, {to_whom}"

> _Note that you can't define Robot syntax and Python in the same cell!_

In [ ]:
*** Settings ***
Library  MyLibrary

*** Test Cases ***
Use my library
    ${greeting} =  Get Greeting
    Log  ${greeting}

Or even import IPython notebooks, like [MyOtherLibrary](./examples/MyOtherLibrary.ipynb), which is inside a subdirectory with an `__init__.py`.

In [ ]:
*** Settings ***
Library  examples.MyOtherLibrary

*** Test Cases ***
My Other Library is also...
    ${greeting} =  Get Greeting
    Log  ${greeting}
    ${farewell} =  Get Farewell
    Log  ${farewell}

### Resources
In addition to keywords defined in python, you can also reuse `.robot` files and `.ipynb` as **[Resources](http://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#resource-and-variable-files)**. Resources **Settings** and **Variables** will become available, but won't execute any **Test Cases** or **Tasks**.

In [ ]:
*** Settings ***
Resource  examples/MyOtherResource.ipynb
Resource  examples/MyResource.robot

*** Test Cases ***
Asking questions
   ${answer} =  Ask The Question  
   ...    How many roads must a small, furry creature from Andromeda walk down?
   Should Be Equal As Numbers  ${answer}  42

### Rich Output
One of the most important components of the Jupyter ecosystem are the messages and client implementations of [rich output](https://ipython.readthedocs.io/en/stable/interactive/plotting.html#rich-outputs). These allow for pusbing visual and interactive components to clients without knowing much at all about HTML, CSS, and JavaScript.

#### Basics
Using `IPython.display` as a [Library](http://robotframework.org/robotframework/latest/RobotFrameworkUserGuide.html#using-test-libraries), we gain all of the rich display powers of IPython. Note that these appear in the _output_ of the notebook, not in the Robot Log.

In [ ]:
*** Settings ***
Library  IPython.display

*** Variables ***
${x}  1
${y}  2
&{point}  x=${x}  y=${y}

*** Test Cases ***
The Two Worlds
    Display HTML  <h1>This goes in the notebook</h1>  raw=True
    Log  <h1>This goes in the report</h1>  html=True
    Run Keyword If  ${x} == ${1}
    ...  Display Markdown  > Markdown works:\n\- x: ${x}\n- y: ${y}</h2>  raw=True
    Display JSON  ${point}  raw=True
    Log  ${point}

#### Widgets
Beyond basic visualization, [Jupyter Widgets](http://jupyter.org/widgets) can create entire interactive applications with just a few lines of code.

> Example TBD

## CLI
Now that you have some Robot notebooks, you can run them at the command line with [nbconvert](https://github.com/jupyter/nbconvert). We're going to use the powerful **[Process](http://robotframework.org/robotframework/latest/libraries/Process.html)** library to call `nbconvert`, and transform our example resource into HTML after `--execute`ing all of the Robot Tasks it contains.

In [ ]:
*** Settings ***
Library  Process

*** Test Cases ***
Convert a Robot Notebook and stop on errors
    ${res} =  Run Process    jupyter  nbconvert  examples/MyOtherResource.ipynb  --execute
    ...  --output  ../_static/DoesntGetCreated
    Should Be Equal As Integers  ${res.rc}  0  msg=${res.stderr}

Oh no! A **Task** failed, but didn't generate any HTML. In the command line, `nbconvert` would have returned a non-zero result, so would stop further execution. Usually, this is the behavior you _want_, as interactively-developed behaviors often build on each other in a procedural fashion. However, not getting your report out is pretty bad.

**However**, if since we are _expecting_ errors in this document, and still want to run everything, you can add `--ExecutePreprocessor.allow_errors=True` to continue after encountering the first error. Also, we set the `--output AllMyOtherResource` so that we can see the difference.

In [ ]:
*** Settings ***
Library  Process

*** Test Cases ***
Convert a Robot Notebook and ignore errors
    ${res} =  Run Process    jupyter  nbconvert  examples/MyOtherResource.ipynb  --execute
    ...    --output  ../_static/AllMyOtherResource  --execute  --ExecutePreprocessor.allow_errors\=True
    Should Be Equal As Integers  ${res.rc}  0  msg=${res.stderr}

Now we can see the failing task and the succcesful one in this [HTML](../_static/examples/AllMyOtherResource.html). In the future, other approaches might be available, such as a more selective converter.

## nbsphinx

[nbsphinx](https://github.com/spatialaudio/nbsphinx) provides a number of neat ways to use notebooks as part of a larger [sphinx](https://www.sphinx-doc.org) documentation suite. Sphinx can be a bit imposing at first, but can generate really nice documentation, along with pre-executed notebooks, or better yet as part of your overall testing procedure.